In [62]:
import pandas as pd
import re


In [63]:
# Load the dataset
file_path = 'vivno_dataset.csv'
try:
    data = pd.read_csv(file_path, encoding='utf-16')
    first_rows = data.head()
    read_success = True
except Exception as e:
    read_success = False
    error_message = str(e)

first_rows if read_success else error_message


,Names,color_wine,Prices,ML,Ratings,Ratingsnum,Countrys,ABV %,rates
0,00 Wines VGW Chardonnay 2017,White Wine,79.99$,750,0.0,0,"Chardonnay from Willamette Valley, Oregon",13.0,"JS, WE, WS"
1,00 Wines VGR Pinot Noir 2019,Red Wine,99.99$,750,0.0,0,"Pinot Noir from Willamette Valley, Oregon",12.9,"WE, JS, RP"
2,00 Wines EGW Chardonnay 2019,White Wine,115.99$,750,0.0,0,"Chardonnay from Willamette Valley, Oregon",0.0,"JS, RP, WE"
3,100 Nails Ranch Pinot Noir 2019,Red Wine,16.99$,750,0.0,0,"Pinot Noir from Central Coast, California",14.2,WW
4,100 Nails Ranch Chardonnay 2019,White Wine,16.99$,750,0.0,0,"Chardonnay from Sonoma County, California",14.2,0


In [64]:

# Extracting the year from the 'Names' column (assuming year is at the end of the string)
data['Year'] = data['Names'].str.extract(r'(\d{4})$')

# Cleaning the 'Prices' column
data['Prices'] = data['Prices'].replace('[\$,]', '', regex=True).astype(float)

# Renaming columns for clarity
data.rename(columns={'ML': 'Volume_ml', 'Ratingsnum': 'Rating_Count', 'Countrys': 'Country_Region'}, inplace=True)


In [65]:
# Handling Missing Data
missing_data_summary = data.isnull().sum()

# Checking for Duplicates
duplicate_rows = data.duplicated().sum()

# Exploring Data Types
data_types = data.dtypes

cleaning_summary = {
    "Missing Data Summary": missing_data_summary,
    "Duplicate Rows": duplicate_rows,
    "Data Types": data_types
}
cleaning_summary


{'Missing Data Summary': Names                0
 color_wine           0
 Prices               0
 Volume_ml            0
 Ratings              0
 Rating_Count         0
 Country_Region       0
 ABV %                0
 rates                0
 Year              1090
 dtype: int64,
 'Duplicate Rows': 0,
 'Data Types': Names              object
 color_wine         object
 Prices            float64
 Volume_ml           int64
 Ratings           float64
 Rating_Count        int64
 Country_Region     object
 ABV %             float64
 rates              object
 Year               object
 dtype: object}

In [66]:

# Splitting the 'Country_Region' column into 'Grape_Varietal' and 'Country_Region'
split_data = data['Country_Region'].str.split(' from ', expand=True)
data['Grape_Varietal'] = split_data[0]
data['Country_Region'] = split_data[1]

# Creating binary columns for raters
raters = ['JS', 'RP', 'WE', 'WS', 'JD', 'D', 'WW', 'V', 'W&S', 'BH', 'TP', 'JM', 'JH', 'SJ', 'CG']
for rater in raters:
    data[rater] = data['rates'].str.contains(rater, regex=False).fillna(False).astype(int)


In [67]:

# Replacing null values in the 'Year' column with "Non-Vintage"
data['Year'].fillna('Non-Vintage', inplace=True)

# Removing instances of 'PR' in the 'RATES' column because possible annomaly identified
data['rates'] = data['rates'].str.replace('PR', '').str.replace(',,', ',').str.strip(',')

# Renaming columns
column_rename_map = {
    "JS": "James_Suckling",
    "RP": "Robert_Parker",
    "WE": "Wine_Enthusiast",
    "WS": "Wine_Spectator",
    "JD": "Jeb_Dunnuck",
    "D": "Decanter",
    "WW": "Wilfred_Wong",
    "V": "Vinous",
    "W&S": "Wine_Spirit",
    "BH": "Burghound",
    "TP": "Tasting_Panel",
    "JM": "Jasper_Morris",
    "JH": "James_Halliday",
    "SJ": "Somm_Journal",
    "CG": "Connoisseurs_Guide",
    "ST": "Stephan_Tanzer",
    "WH": "Whiskey_Advocate"
}
data.rename(columns=column_rename_map, inplace=True)

# Capitalizing all column names
data.columns = data.columns.str.upper()


In [68]:
# Handling Missing Data
missing_data_summary = data.isnull().sum()

# Checking for Duplicates
duplicate_rows = data.duplicated().sum()

# Exploring Data Types
data_types = data.dtypes

cleaning_summary = {
    "Missing Data Summary": missing_data_summary,
    "Duplicate Rows": duplicate_rows,
    "Data Types": data_types
}
cleaning_summary


{'Missing Data Summary': NAMES                  0
 COLOR_WINE             0
 PRICES                 0
 VOLUME_ML              0
 RATINGS                0
 RATING_COUNT           0
 COUNTRY_REGION        40
 ABV %                  0
 RATES                  0
 YEAR                   0
 GRAPE_VARIETAL         0
 JAMES_SUCKLING         0
 ROBERT_PARKER          0
 WINE_ENTHUSIAST        0
 WINE_SPECTATOR         0
 JEB_DUNNUCK            0
 DECANTER               0
 WILFRED_WONG           0
 VINOUS                 0
 WINE_SPIRIT            0
 BURGHOUND              0
 TASTING_PANEL          0
 JASPER_MORRIS          0
 JAMES_HALLIDAY         0
 SOMM_JOURNAL           0
 CONNOISSEURS_GUIDE     0
 dtype: int64,
 'Duplicate Rows': 0,
 'Data Types': NAMES                  object
 COLOR_WINE             object
 PRICES                float64
 VOLUME_ML               int64
 RATINGS               float64
 RATING_COUNT            int64
 COUNTRY_REGION         object
 ABV %                 float64
 

In [69]:
# Analyzing patterns in the 'NAMES' column for rows where 'COUNTRY_REGION' is missing
missing_country_data = data[data['COUNTRY_REGION'].isnull()]
names_with_missing_country = missing_country_data['NAMES']

# Creating mapping rules based on observed patterns in wine names
# Note: These are basic examples and might not cover all cases

# Example mapping rules
mapping_rules = {
    #General rules observed
    'Champagne': 'Champagne, France',
    'Napa Valley': 'Napa Valley, USA',
    'Marcassin Vineyard': 'California, USA',
    'Pride Mountain Vineyards': 'California, USA',
    'Spottswoode': 'California, USA',
    # USA - California specific vineyards and wineries
    'Bevan Cellars': 'California, USA',
    'Black Stallion Winery': 'California, USA',
    'Bryant Family': 'California, USA',
    'Hundred Acre Kayli Morgan': 'California, USA',
    'To Kalon Vineyard': 'Napa Valley, USA',
    'Screaming Eagle': 'Napa Valley, USA',
    'Sine Qua Non': 'California, USA',
    # France specific regions and vineyards
    'Chateau Montrose': 'Bordeaux, France',
    'Chateau Mouton Rothschild': 'Bordeaux, France',
    'Chambolle-Musigny': 'Burgundy, France'
}

# Function to apply mapping rules
def infer_country_region(name, mapping_rules):
    for keyword, region in mapping_rules.items():
        if keyword in name:
            return region
    return "Unknown"

# Apply the mapping rules to the 'NAMES' column where 'COUNTRY_REGION' is missing
inferred_country_regions = names_with_missing_country.apply(lambda name: infer_country_region(name, mapping_rules))

# Filling in the missing 'COUNTRY_REGION' values with the inferred values
data.loc[data['COUNTRY_REGION'].isnull(), 'COUNTRY_REGION'] = inferred_country_regions

# Checking how many missing values are remaining after this process
remaining_missing_values = data['COUNTRY_REGION'].isnull().sum()
remaining_missing_values


0

In [70]:
# Handling Missing Data
missing_data_summary = data.isnull().sum()

# Checking for Duplicates
duplicate_rows = data.duplicated().sum()

# Exploring Data Types
data_types = data.dtypes

cleaning_summary = {
    "Missing Data Summary": missing_data_summary,
    "Duplicate Rows": duplicate_rows,
    "Data Types": data_types
}
cleaning_summary

{'Missing Data Summary': NAMES                 0
 COLOR_WINE            0
 PRICES                0
 VOLUME_ML             0
 RATINGS               0
 RATING_COUNT          0
 COUNTRY_REGION        0
 ABV %                 0
 RATES                 0
 YEAR                  0
 GRAPE_VARIETAL        0
 JAMES_SUCKLING        0
 ROBERT_PARKER         0
 WINE_ENTHUSIAST       0
 WINE_SPECTATOR        0
 JEB_DUNNUCK           0
 DECANTER              0
 WILFRED_WONG          0
 VINOUS                0
 WINE_SPIRIT           0
 BURGHOUND             0
 TASTING_PANEL         0
 JASPER_MORRIS         0
 JAMES_HALLIDAY        0
 SOMM_JOURNAL          0
 CONNOISSEURS_GUIDE    0
 dtype: int64,
 'Duplicate Rows': 0,
 'Data Types': NAMES                  object
 COLOR_WINE             object
 PRICES                float64
 VOLUME_ML               int64
 RATINGS               float64
 RATING_COUNT            int64
 COUNTRY_REGION         object
 ABV %                 float64
 RATES                  obj

In [71]:
# Replacing any value in the 'GRAPE_VARIETAL' column that contains any form of the word "Collection" with "Unknown Blend"
data['GRAPE_VARIETAL'] = data['GRAPE_VARIETAL'].apply(lambda x: "Unknown Blend" if "collection" in x.lower() else x)

# Check if the replacement was successful
data['GRAPE_VARIETAL'].value_counts()


GRAPE_VARIETAL
Pinot Noir             1706
Cabernet Sauvignon     1498
Bordeaux Red Blends    1386
Chardonnay             1338
Other Red Blends        816
                       ... 
Grolleau                  1
Verdelho                  1
Poulsard                  1
White Zinfandel           1
Loureiro                  1
Name: count, Length: 131, dtype: int64

In [72]:

# # Export the cleaned dataset to an Excel file
# cleaned_file_path = 'M06_FinalProject_CleanedDataset_MGentile.xlsx'
# data.to_excel(cleaned_file_path, index=False)
